# Imbalanced classification: credit card fraud detection

In [2]:
import csv
import numpy as np

### get data from kaggle https://www.kaggle.com/mlg-ulb/creditcardfraud/

In [6]:
fname = "C:/Users/Safouane Elh/Documents/MBD S3/Deep Learning/Atelier 1/creditcard.csv"

## First, vectorize the CSV data


In [7]:
all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])
features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)   

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


## Prepare a validation set



In [8]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


## Analyze class imbalance in the targets



In [9]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


## Normalize the data using training set statistics


### This improves the accuracy and integrity of our data, while making our database easier to navigate. 

In [10]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

## Build a binary classification model


In [11]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7936      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 139,777
Trainable params: 139,777
Non-trai

## Train the model with class_weight argument  to adjust such setting


In [12]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/30
112/112 - 12s - loss: 2.4753e-06 - fn: 47.0000 - fp: 25521.0000 - tn: 201908.0000 - tp: 370.0000 - precision: 0.0143 - recall: 0.8873 - val_loss: 0.3125 - val_fn: 6.0000 - val_fp: 5240.0000 - val_tn: 51646.0000 - val_tp: 69.0000 - val_precision: 0.0130 - val_recall: 0.9200 - 12s/epoch - 112ms/step
Epoch 2/30
112/112 - 6s - loss: 1.4141e-06 - fn: 34.0000 - fp: 6628.0000 - tn: 220801.0000 - tp: 383.0000 - precision: 0.0546 - recall: 0.9185 - val_loss: 0.0425 - val_fn: 11.0000 - val_fp: 327.0000 - val_tn: 56559.0000 - val_tp: 64.0000 - val_precision: 0.1637 - val_recall: 0.8533 - 6s/epoch - 54ms/step
Epoch 3/30
112/112 - 6s - loss: 1.1416e-06 - fn: 34.0000 - fp: 6140.0000 - tn: 221289.0000 - tp: 383.0000 - precision: 0.0587 - recall: 0.9185 - val_loss: 0.0899 - val_fn: 9.0000 - val_fp: 1282.0000 - val_tn: 55604.0000 - val_tp: 66.0000 - val_precision: 0.0490 - val_recall: 0.8800 - 6s/epoch - 53ms/step
Epoch 4/30
112/112 - 6s - loss: 1.0708e-06 - fn: 21.0000 - fp: 7355.0000 - tn:

Epoch 28/30
112/112 - 6s - loss: 3.3506e-07 - fn: 3.0000 - fp: 2810.0000 - tn: 224619.0000 - tp: 414.0000 - precision: 0.1284 - recall: 0.9928 - val_loss: 0.1155 - val_fn: 5.0000 - val_fp: 2667.0000 - val_tn: 54219.0000 - val_tp: 70.0000 - val_precision: 0.0256 - val_recall: 0.9333 - 6s/epoch - 51ms/step
Epoch 29/30
112/112 - 6s - loss: 3.0571e-07 - fn: 2.0000 - fp: 3534.0000 - tn: 223895.0000 - tp: 415.0000 - precision: 0.1051 - recall: 0.9952 - val_loss: 0.0212 - val_fn: 8.0000 - val_fp: 406.0000 - val_tn: 56480.0000 - val_tp: 67.0000 - val_precision: 0.1416 - val_recall: 0.8933 - 6s/epoch - 52ms/step
Epoch 30/30
112/112 - 6s - loss: 2.2508e-07 - fn: 3.0000 - fp: 2746.0000 - tn: 224683.0000 - tp: 414.0000 - precision: 0.1310 - recall: 0.9928 - val_loss: 0.0132 - val_fn: 12.0000 - val_fp: 207.0000 - val_tn: 56679.0000 - val_tp: 63.0000 - val_precision: 0.2333 - val_recall: 0.8400 - 6s/epoch - 50ms/step


## Conclusion
At the end of training, out of 56,961 validation transactions, we are:

I see how we can correctly identifying 66 of them as fraudulent
Missing 9 fraudulent transactions
At the cost of incorrectly flagging 441 legitimate transactions